# `2.1 Bitstring Generation: Creating Configurations For The Flight Code 🚀`

## <span style="color:tomato"> Overview </span>

In this section, you will essentially be creating a dataset that can be passed into the flight code. 

#### <span style="color:orange">1. Bitstrings </span>

The first item in this dataset will be a list of bitstrings. A bitstring is a string of binary digits (0s and 1s) that can be interpreted as an integer value. You may be familiar with binary numbers, where each digit is equal to its value (1 or 0) times $2^i$, where $i$ is the position of the digit relative to the right end of the number. The value of a binary number is the sum of its digits' values times $2^i$.

<span style="color:orange">$$ Value = \sum_{i=0}^{n-1} b_i * 2^i $$</span>
where <span style="color:orange">$b_i$</span> is the ith binary digit from the right and <span style="color:orange">$n$</span> is the number of digits.

A bitstring does not necessarily represent a number, but instead information that can be easily encoded with 0s and 1s, like our configurations. The advantage of using a bitstring over other datatypes, such as integers, strings, or decimal numbers, is that it condenses information and saves space, as long as program logic is in place to properly interpret what different parts of the bitstring means. Each configurable LunaSat component will contribute to a separable part of the bitstring such that you could look at just that portion of the bitstring to identify the configuration of the particular component.

The format of the list of bitstrings can be seen below:

<span style="color:orange"> [Bitstring 1, Bitstring 2, Bitstring 3, ... ] </span>

<span style="color:orange"> Bitstring # =  "0b| TP Mode | CAP Mode | TMP No. Averages | TMP Conv. Cycle Time | TMP Mode | ACC Low Power | ACC Dig. Low Pass | ACC SRD | ACC Mode | Mag No. Averages | Mag Freq. | MAG Mode |" </span>

<span style="color:orange"> Ex: Bitstring 1 = "0b10000001010001010000000000001110001" </span>


#### <span style="color:orange">2. Durations </span>

The second item in this dataset will be a list of configuration durations. The ith duration in this list will correspond to the ith bitstring in the first list. The list will be in nested form, where the outer list's elements correspond to bitstrings while the inner lists' elements correspond to sensors and components. The format of this list can be seen below:

<span style="color:orange"> [[Durations 1], [Durations 2], [Durations 3], ... ] </span>

<span style="color:orange"> [Durations #] = [TP,CAP,TMP,ACC,MAG] </span>

<span style="color:orange"> Ex: TP = 60, CAP = 5, etc. </span>

#### <span style="color:orange">3. Sampling Rates </span>

The third item in this dataset will be a list of sampling rates. Each sampling rate should be a decimal number and should conform to the capabilities of the appropriate sensors as well as the overarching data rate for the LunaSat. The list will be in nested form, where the outer list's elements correspond to bitstrings while the inner lists' elements correspond to sensors and components. The structure of this list can be seen below:


<span style="color:orange"> [[Sampling Rates 1],[Sampling Rates 2],[Sampling Rates 3],...] </span>

<span style="color:orange"> [Sampling Rates #] = [TP,CAP,TMP,ACC,MAG] </span>

<span style="color:orange"> Ex: TP = 0.1, CAP = 2, etc. </span>


## <span style="color:tomato"> Setting Configurations </span>

<span style="color:#18BF7D">

### Please read these instructions very carefully.

Below, you will be able to rewrite the configurations you chose before. If you wish to create configurations different from those you created previously, be sure to check that your new configurations adhere to mission requirements. We strongly encourage you to go back to each notebook for the components you wish to change configurations for and re-run all cells to ensure that the power/data usage are acceptable. We also strongly encourage that you revisit "2.0 Combined PDM" once you do this to verify that the configurations do not exceed power and data limitations. Functions will be used to check that your configurations are valid. If you do not do this, and you generate configurations that do not meet our requirements, we cannot guarantee that your configurations will make it onto the mission. If you are not changing your configurations, simply copy and paste the relevant parameters from each notebook into the code in this notebook. Note that the format of the configurations in this notebook is slightly different than before. For example, the configuration durations are no longer unique to each sensor, but to the overall configurations themselves. Switching sensor modes will be done simultaneously between configurations rather than one sensor at a time.
    
If your configurations are valid, you will be able to generate the dataset. We recommend minimizing the number of unique configurations you create for each sensor. 3 is a good number, but you may define more if you wish. Each sensor should have the same number of configurations. If you do not wish a sensor's configuration to change while others do change, simply repeat its configuration such that no changes to settings are needed. We also recommend choosing configuration durations such that they are considerably long, as switching configurations may require significant setup and downtime while code is executing. Setup may require clearing memory and redefining memory partitions if sensors are turned on and/or off. Since code must run synchronously, this will result in a delay to collecting data or performing other operations.
    
You may assume for now that these configurations will repeat indefinitely until RF is turned on and all sensors are turned off. If the total length of all your durations is ultimately less than the time allotted for data sampling, your configurations will loop such that once the last one finishes, the first one starts again. If the total length of all your durations is ultimately longer than the time allotted for data sampling, your configurations will be interrupted. It has not yet been defined whether data sampling will restart with the most recent configuration or the first configuration once RF is turned off. 
</span>

This is currently outside the scope of the LPDM, but you should be aware than code cannot execute instantaneously, and the time it takes to reach the lines of code for collecting a sample from a particular sensor is dependent on how large the script is. This is another factor that has not yet been finalized, but ultimately the sensors will be sampled at either your defined rates or the rate at which the code can complete one loop iteration, whichever is slower.

## <span style="color:tomato"> Example Configuration Dataset </span>

<span style="color:#18BF7D">Do not modify the following code. Simply run the cell to see the output.</span>

In [ ]:
import micropip
await micropip.install("matplotlib")
import matplotlib.pyplot as plt
await micropip.install("numpy")
import numpy as np
from source.helperFunctions import *
from source.generateBitstrings import *

# In this example, only one configuration (consisting of one bitstring, one duration, and one set of sampling rates) is chosen

team_name = 'EXAMPLE_ARDUINO_CONFIGS' # Your team name
team_no = 0 # Your team number

duration_list = [600] # seconds

# TMP - (mode, num_averages, conv_cycle_time)
config_TMP = ("ONE_SHOT", 0, 0.0155)
sampling_TMP = 10 # seconds

# MAG - (mode, sample_freq, num_averages) 
config_MAG = ("SINGLE", 1000, 4)
sampling_MAG = 30 # seconds

# ACC - (mode, low_power_wakeup, digital_low_pass, sampling rate divisor)
config_ACC = ("ACCELEROMETER", 0, "101", 0)
sampling_ACC = 0.001 # seconds

# TP - (mode)
config_TP = ("TP_OFF")
sampling_TP = 10 # seconds

# CAP - (mode)
config_CAP = ("CAP_ON")
sampling_CAP = 250 # seconds

config_list = [[config_TP, config_CAP, config_TMP, config_ACC, config_MAG]]
sampling_rates_list = [[sampling_TP, sampling_CAP, sampling_TMP, sampling_ACC, sampling_MAG]]

valid = validate_configs(config_list, sampling_rates_list, duration_list)
if(valid):
    generate_dataset(config_list, duration_list, sampling_rates_list, team_name, team_no)
    divide_bitstring()

## <span style="color:tomato"> Create Your Configurations Here </span>

<span style="color:#18BF7D">For the following code, replace 'None' with appropriate values. If you wish to create more than 3 configurations, copy the section surrounded by these marks: </span>
    
<span style="color:#18BF7D">**#--!--!--!#** </span>
    
<span style="color:#18BF7D">Paste it in the same cell at the very end. </span>

In [ ]:
# SET YOUR TEAM NAME AND TEAM NUMBER HERE
team_name = None
team_no = None

In [ ]:
config_list = []
duration_list = []
sampling_rates_list = []

# Below are 3 sections for adding your configurations

# CONFIGURATION 1:
duration_list.append(None) # duration

config_TMP = None
sampling_TMP = None

config_MAG = None
sampling_MAG = None

config_ACC = None
sampling_ACC = None

config_TP = None
sampling_TP = None

config_CAP = None
sampling_CAP = None

config_list.append([config_TP, config_CAP, config_TMP, config_ACC, config_MAG]) # configurations
sampling_rates_list.append([sampling_TP, sampling_CAP, sampling_TMP, sampling_ACC, sampling_MAG]) # sampling rates

# CONFIGURATION 2:
duration_list.append(None) # duration

config_TMP = None
sampling_TMP = None

config_MAG = None
sampling_MAG = None

config_ACC = None
sampling_ACC = None

config_TP = None
sampling_TP = None

config_CAP = None
sampling_CAP = None

config_list.append([config_TP, config_CAP, config_TMP, config_ACC, config_MAG]) # configurations
sampling_rates_list.append([sampling_TP, sampling_CAP, sampling_TMP, sampling_ACC, sampling_MAG]) # sampling rates

#--!--!--!#

# CONFIGURATION 3:
duration_list.append(None) # duration

config_TMP = None
sampling_TMP = None

config_MAG = None
sampling_MAG = None

config_ACC = None
sampling_ACC = None

config_TP = None
sampling_TP = None

config_CAP = None
sampling_CAP = None

config_list.append([config_TP, config_CAP, config_TMP, config_ACC, config_MAG]) # configurations
sampling_rates_list.append([sampling_TP, sampling_CAP, sampling_TMP, sampling_ACC, sampling_MAG]) # sampling rates

#--!--!--!#

<span style="color:#18BF7D">Run the following code to see if your configurations are valid. If you receive an error, go back to the cell above and address it. You will not be able to generate the dataset until all configurations are valid. </span>

In [ ]:
valid = validate_configs(config_list, sampling_rates_list, duration_list) # VALIDATING CONFIGS

If your configurations are valid, you will be able to run the code below to generate the dataset of bitstrings, durations, and sampling rates. The dataset will be outputted both in this dataset and in a text file called '{team_no}_{team_name}.txt'. You may send this text file to GLEE's email so that we can include it in our flight code.

<span style="color:#18BF7D">Do not modify the following code. Simply run the cell to generate the dataset.</span>

In [ ]:
# GENERATE BITSTRINGS FOR VALID MODES IF INPUTS ARE VALID
if valid == True:
    generate_dataset(config_list, duration_list, sampling_rates_list, team_name, team_no)
else: print("Dataset Generation Failed!")
    
# The generated bitstring is a binary representation of the configurations you have created. This bitstring will be used as input to the lunar code.

[Click here to continue to the final LPDM notebook.](3.0%20Next%20Steps.ipynb)